In [ ]:
'''
keras
tensorflow
pillow
scikit-image
h5py
'''

# START

In [1]:
# create a data generator
import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Conv2D
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from PIL import Image
import numpy as np
from skimage import transform
import os
import shutil
from pathlib import Path


Using TensorFlow backend.


In [2]:
# prepare an iterators for each dataset
datagen = ImageDataGenerator()
data = datagen.flow_from_directory('train/', class_mode='binary')

Found 15323 images belonging to 7 classes.


In [3]:
totalBadBG = len([name for name in os.listdir('train/Bad_Backgroud') if os.path.isfile(os.path.join('train/Bad_Backgroud', name))])
totalBlurry = len([name for name in os.listdir('train/Blurry') if os.path.isfile(os.path.join('train/Blurry', name))])
totalFaceBG = len([name for name in os.listdir('train/Face_in_Background') if os.path.isfile(os.path.join('train/Face_in_Background', name))])
totalGood = len([name for name in os.listdir('train/Good_Images') if os.path.isfile(os.path.join('train/Good_Images', name))])
totalInvalidP = len([name for name in os.listdir('train/Invalid_Passport') if os.path.isfile(os.path.join('train/Invalid_Passport', name))])
totalNoHumanFace = len([name for name in os.listdir('train/No_Human_Face') if os.path.isfile(os.path.join('train/No_Human_Face', name))])
totalStapleDefaced = len([name for name in os.listdir('train/Staple_Defaced') if os.path.isfile(os.path.join('train/Staple_Defaced', name))])

print(f'Total number of bad background images {totalBadBG}')
print(f'Total number of blurry images {totalBlurry}')
print(f'Total number of face in background images {totalFaceBG}')
print(f'Total number of good images {totalGood}')
print(f'Total number of invalid passport images {totalInvalidP}')
print(f'Total number of human faces in bg images {totalNoHumanFace}')
print(f'Total number of staple defaced images {totalStapleDefaced}')

Total number of bad background images 1690
Total number of blurry images 700
Total number of face in background images 89
Total number of good images 6240
Total number of invalid passport images 786
Total number of human faces in bg images 0
Total number of staple defaced images 10


# STOP

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    "train/",
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    "train/", # same directory as training data
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 12262 images belonging to 7 classes.
Found 3061 images belonging to 7 classes.


In [5]:
from keras.applications.vgg16 import VGG16
model = VGG16()

In [6]:
print(model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
from keras.models import Model
from keras import models
model1  = Model(inputs = model.input, output = model.get_layer('block4_conv1').output)
model1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

/opt/anaconda3/envs/notebook-5.7.8/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
Model1 = models.Sequential()
Model1.add(model1)
from keras.layers import Conv2D, Dense, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
Model1.add(Conv2D(128, (3,3), activation='relu', padding = 'same'))
Model1.add(MaxPooling2D((2,2), padding = 'same'))
Model1.add(GlobalAveragePooling2D())
Model1.add(Dense(64, activation = 'relu'))
Model1.add(Dense(7, activation = 'softmax'))
Model1.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 28, 28, 512)       2915648   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       589952    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 455       
Total params: 3,514,311
Trainable params: 3,514,311
Non-trainable params: 0
____________________________________________

In [9]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
Model1.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])


In [10]:
Model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 28, 28, 512)       2915648   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       589952    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 455       
Total params: 3,514,311
Trainable params: 3,514,311
Non-trainable params: 0
____________________________________________

In [11]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [12]:
from PIL import Image
print(Image.__file__)

/opt/anaconda3/envs/notebook-5.7.8/lib/python3.7/site-packages/PIL/Image.py


In [13]:
import Image
print(Image.__file__)

/opt/anaconda3/envs/notebook-5.7.8/lib/python3.7/site-packages/PIL/Image.py


# START

In [14]:
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)
checkpoint_callback = ModelCheckpoint("model2"+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# STOP

In [15]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)
checkpoint_callback = ModelCheckpoint("model1"+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [15]:
Model1.fit_generator(train_generator, steps_per_epoch=32, epochs = 40, validation_data=validation_generator, validation_steps=8, callbacks=[early_stopping_callback, checkpoint_callback])






Epoch 1/40
32/32 [==============================] - 1187s 37s/step - loss: 1.8802 - accuracy: 0.7192 - val_loss: 0.7671 - val_accuracy: 0.8125

Epoch 00001: val_loss improved from inf to 0.76711, saving model to model1.h5
Epoch 2/40
32/32 [==============================] - 1149s 36s/step - loss: 0.7630 - accuracy: 0.7933 - val_loss: 0.8731 - val_accuracy: 0.7637

Epoch 00002: val_loss did not improve from 0.76711
Epoch 3/40
32/32 [==============================] - 1136s 35s/step - loss: 0.7536 - accuracy: 0.7646 - val_loss: 0.6796 - val_accuracy: 0.7754

Epoch 00003: val_loss improved from 0.76711 to 0.67958, saving model to model1.h5
Epoch 4/40
32/32 [==============================] - 1149s 36s/step - loss: 0.5928 - accuracy: 0.8081 - val_loss: 0.4012 - val_accuracy: 0.8145

Epoch 00004: val_loss improved from 0.67958 to 0.40122, saving model to model1.h5
Epoch 5/40
32/32 [==============================] - 1148s 36s/step - loss: 0.5280 - accuracy: 0.8276 - val_loss: 0.4735 - val_accur

In [16]:
Model1.save('model.h5')

In [29]:
from keras.models import load_model
def fix_layer0(filename, batch_input_shape, dtype):
    import h5py
    import json
    
    with h5py.File(filename, 'r+') as f:
        model_config = json.loads(f.attrs['model_config'].decode('utf-8'))
        layer0 = model_config['config']['layers'][0]['config']
        layer0['batch_input_shape'] = batch_input_shape
        layer0['dtype'] = dtype
        f.attrs['model_config'] = json.dumps(model_config).encode('utf-8')

fix_layer0('model.h5', [None, 224, 224, 3], 'float32')

loaded_model = load_model('model.h5')


# MAKE PREDICTIONS PER FOLDERS

In [44]:
test_gen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_generator = test_gen.flow_from_directory(
        "test/",
        target_size=(224, 224),
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

Found 213 images belonging to 2 classes.


In [100]:
from PIL import Image
import numpy as np
from skimage import transform

def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (224, 224, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image



In [143]:
def startPrediction(stateName):
    import os
    import shutil
    from pathlib import Path
    test_gen = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    currentList = f'{stateName}/'
    directoryList = os.listdir(currentList)
    #loop through the centers
    current = 0
    total = len([name for name in os.listdir(currentList) if os.path.isfile(os.path.join(currentList, name))])
    total = sum([len(files) for r, d, files in os.walk(currentList)])
    for directory in directoryList:
        print(directory)
        if directory != '.DS_Store':
            currentImageDirectory = f'{currentList}{directory}'
            for subdir, dirs, files in os.walk(currentImageDirectory):
                print(currentImageDirectory)
                
                for file in files:
                    filepath = subdir + os.sep + file
                    if filepath.endswith(".jpg"):
                        image = load(filepath)
                        _class = Model1.predict_classes(image)
                        if _class == 0:
                            #BAD BACKGROUND
                            if os.path.exists(f"{currentImageDirectory}/BAD_BACKGROUND") == True:
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/BAD_BACKGROUND")
                                except:
                                    error = 1
                            else:
                                #make directory
                                os.mkdir(f"{currentImageDirectory}/BAD_BACKGROUND")
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/BAD_BACKGROUND")
                                except:
                                    error = 1
                            current +=1
                                    
                        elif _class == 1:
                            #BLURRY
                            if os.path.exists(f"{currentImageDirectory}/BLURRY") == True:
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/BLURRY")
                                except:
                                    error = 1
                            else:
                                #make directory
                                os.mkdir(f"{currentImageDirectory}/BLURRY")
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/BLURRY")
                                except:
                                    error = 1
                            current +=1
                                    
                        elif _class == 2:
                            #FACE_IN_BACKGROUND
                            if os.path.exists(f"{currentImageDirectory}/FACE_IN_BACKGROUND") == True:
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/FACE_IN_BACKGROUND")
                                except:
                                    error = 1
                            else:
                                #make directory
                                os.mkdir(f"{currentImageDirectory}/FACE_IN_BACKGROUND")
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/FACE_IN_BACKGROUND")
                                except:
                                    error = 1
                            current +=1
                                    
                        elif _class == 3:
                            #GOOD_IMAGES
                            if os.path.exists(f"{currentImageDirectory}/GOOD_IMAGES") == True:
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/GOOD_IMAGES")
                                except:
                                    error = 1
                            else:
                                #make directory
                                os.mkdir(f"{currentImageDirectory}/GOOD_IMAGES")
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/GOOD_IMAGES")
                                except:
                                    error = 1
                            current +=1
                                    
                        elif _class == 4:
                            #INVALID_PASSPORT
                            if os.path.exists(f"{currentImageDirectory}/INVALID_PASSPORT") == True:
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/INVALID_PASSPORT")
                                except:
                                    error = 1
                            else:
                                #make directory
                                os.mkdir(f"{currentImageDirectory}/INVALID_PASSPORT")
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/INVALID_PASSPORT")
                                except:
                                    error = 1
                            current +=1
                                    
                        elif _class == 5:
                            #NO_HUMAN_FACE
                            if os.path.exists(f"{currentImageDirectory}/NO_HUMAN_FACE") == True:
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/NO_HUMAN_FACE")
                                except:
                                    error = 1
                            else:
                                #make directory
                                os.mkdir(f"{currentImageDirectory}/NO_HUMAN_FACE")
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/NO_HUMAN_FACE")
                                except:
                                    error = 1
                            current +=1
                                    
                        elif _class == 6:
                            #STAPLED FACE
                            if os.path.exists(f"{currentImageDirectory}/STAPLED_FACE") == True:
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/STAPLED_FACE")
                                except:
                                    error = 1
                            else:
                                #make directory
                                os.mkdir(f"{currentImageDirectory}/STAPLED_FACE")
                                try:
                                    shutil.move(f"{filepath}" , f"{currentImageDirectory}/STAPLED_FACE")
                                except:
                                    error = 1
                            current +=1
                        
                        print(f'{current} ---of--- {total}')
                              
                        

                
                
                

                
    

In [144]:
startPrediction('438')

4380303
438/4380303
438/4380303
1 ---of--- 214
2 ---of--- 214
3 ---of--- 214
4 ---of--- 214
5 ---of--- 214
6 ---of--- 214
7 ---of--- 214
8 ---of--- 214
9 ---of--- 214
10 ---of--- 214
11 ---of--- 214
12 ---of--- 214
13 ---of--- 214
14 ---of--- 214
15 ---of--- 214
16 ---of--- 214
.DS_Store
4380304
438/4380304
438/4380304
17 ---of--- 214
18 ---of--- 214
19 ---of--- 214
20 ---of--- 214
21 ---of--- 214
22 ---of--- 214
23 ---of--- 214
24 ---of--- 214
25 ---of--- 214
26 ---of--- 214
27 ---of--- 214
28 ---of--- 214
29 ---of--- 214
30 ---of--- 214
31 ---of--- 214
32 ---of--- 214
33 ---of--- 214
34 ---of--- 214
35 ---of--- 214
36 ---of--- 214
37 ---of--- 214
38 ---of--- 214
39 ---of--- 214
40 ---of--- 214
41 ---of--- 214
42 ---of--- 214
43 ---of--- 214
44 ---of--- 214
45 ---of--- 214
46 ---of--- 214
47 ---of--- 214
48 ---of--- 214
49 ---of--- 214
50 ---of--- 214
51 ---of--- 214
52 ---of--- 214
53 ---of--- 214
4380305
438/4380305
438/4380305
54 ---of--- 214
55 ---of--- 214
56 ---of--- 214
57 ---o

# PREDICT ONE IMAGE

In [30]:
test_gen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_generator = test_gen.flow_from_directory(
        "test/",
        target_size=(224, 224),
        shuffle = False,
        class_mode='categorical',
        batch_size=1)





Found 1 images belonging to 1 classes.


In [31]:
predict = loaded_model.predict_generator(test_generator)

In [32]:
array  = np.array(predict)

In [33]:
ind = np.argmax(predict)

In [34]:
predict

array([[4.9596351e-01, 3.0768332e-01, 6.7714512e-02, 2.9282017e-02,
        6.9323264e-02, 3.8860162e-04, 2.9644707e-02]], dtype=float32)

In [35]:
ind+1

1

# CLASSES

In [34]:
Model1.predict_classes(test_generator)

array([[1.9350958e-01, 1.0001611e-02, 2.0744037e-02, 7.2890091e-01,
        4.6525884e-02, 3.8137823e-05, 2.7990725e-04]], dtype=float32)

# USING RESNET50

In [155]:
from keras import applications
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D

In [156]:
img_height,img_width = 64,64
num_classes = 6
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))

/opt/anaconda3/envs/notebook-5.7.8/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [157]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    "train/",
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    "train/", # same directory as training data
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 12262 images belonging to 7 classes.
Found 3061 images belonging to 7 classes.


In [158]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(7, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [159]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [160]:
model.fit_generator(train_generator, steps_per_epoch=32, epochs = 50, validation_data=validation_generator, validation_steps=8, callbacks=[early_stopping_callback, checkpoint_callback])



Epoch 1/50
32/32 [==============================] - 151s 5s/step - loss: 1.1856 - accuracy: 0.6475 - val_loss: 16.0457 - val_accuracy: 0.7832

Epoch 00001: val_loss did not improve from 0.11477
Epoch 2/50
32/32 [==============================] - 128s 4s/step - loss: 0.6329 - accuracy: 0.8136 - val_loss: 1.2818 - val_accuracy: 0.7754

Epoch 00002: val_loss did not improve from 0.11477
Epoch 3/50
32/32 [==============================] - 126s 4s/step - loss: 0.5285 - accuracy: 0.8403 - val_loss: 0.9815 - val_accuracy: 0.6973

Epoch 00003: val_loss did not improve from 0.11477
Epoch 4/50
32/32 [==============================] - 140s 4s/step - loss: 0.5000 - accuracy: 0.8516 - val_loss: 0.6192 - val_accuracy: 0.8379

Epoch 00004: val_loss did not improve from 0.11477
Epoch 5/50
32/32 [==============================] - 155s 5s/step - loss: 0.4805 - accuracy: 0.8716 - val_loss: 0.5409 - val_accuracy: 0.8281

Epoch 00005: val_loss did not improve from 0.11477
Epoch 6/50
32/32 [================

# START

# MODEL 2 WITH DEEPER NETWROKS IN VGG16

In [16]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    "train/",
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    "train/", # same directory as training data
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 12262 images belonging to 7 classes.
Found 3061 images belonging to 7 classes.


In [17]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D, InputLayer
model = VGG16()

In [18]:
from keras.models import Model
from keras import models
model2  = Model(inputs = model.input, output = model.get_layer('fc2').output)
model2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

/opt/anaconda3/envs/notebook-5.7.8/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
Model2 = models.Sequential()
Model2.add(model2)
from keras.layers import Conv2D, Dense, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
Model2.add(Dense(7, activation = 'softmax'))
Model2.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 4096)              134260544 
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 28679     
Total params: 134,289,223
Trainable params: 134,289,223
Non-trainable params: 0
_________________________________________________________________


In [20]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
Model2.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])


In [21]:
Model2.fit_generator(train_generator, steps_per_epoch=32, epochs = 30, validation_data=validation_generator, validation_steps=8, callbacks=[early_stopping_callback, checkpoint_callback])



Epoch 1/30
32/32 [==============================] - 2069s 65s/step - loss: 10.2417 - accuracy: 0.7178 - val_loss: 1.0631 - val_accuracy: 0.7793

Epoch 00001: val_loss improved from inf to 1.06309, saving model to model1.h5
Epoch 2/30
32/32 [==============================] - 1807s 56s/step - loss: 0.8036 - accuracy: 0.7852 - val_loss: 1.0900 - val_accuracy: 0.7910

Epoch 00002: val_loss did not improve from 1.06309
Epoch 3/30
32/32 [==============================] - 1892s 59s/step - loss: 0.7816 - accuracy: 0.7759 - val_loss: 0.4071 - val_accuracy: 0.8125

Epoch 00003: val_loss improved from 1.06309 to 0.40714, saving model to model1.h5
Epoch 4/30
32/32 [==============================] - 1850s 58s/step - loss: 0.5526 - accuracy: 0.8279 - val_loss: 0.3057 - val_accuracy: 0.9043

Epoch 00004: val_loss improved from 0.40714 to 0.30573, saving model to model1.h5
Epoch 5/30
32/32 [==============================] - 1736s 54s/step - loss: 0.3670 - accuracy: 0.9038 - val_loss: 0.3761 - val_accu

In [22]:
Model2.save('model.h5')

# STOP